In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import json
from datetime import datetime

from data_helpers.syntax_graph import SyntaxGraph
from data_helpers.nc21_reader import Nc21Reader

# functions for creating database and collecting collocations
from collect_functions_1 import *


# Tabel 1 andmete kogumine

In [2]:
%%time
# takes ca 2 mins
# file_name = 'data/nc21_Fiction.vert_10'
file_name = 'data/balanced_test.vert_10'

my_nc21_reader = Nc21Reader(
   file_name = file_name
)

CPU times: user 58 µs, sys: 1 µs, total: 59 µs
Wall time: 63.2 µs


In [ ]:
"""
for sentence_id, text in my_nc21_reader.get_sentences():
    print(sentence_id)
    text.draw_graph()
    break
"""

TYPE = 'obl_verb'
TABLENAME = f'{TYPE}'
BATCH_SIZE = 1

date_time = datetime.now().strftime("%Y%m%d-%H%M%S")
db_file_name = f"balanced_test_{TYPE}_collocations_{date_time}.db"
my_sqlite_db = DbMethods(db_file_name=db_file_name, table1_name=TYPE, table2_name=TYPE+'_examples')
my_sqlite_db.prep_coll_db()


collocations = {}
count = 0
for collection_id, graph in my_nc21_reader.get_sentences():
    
    count += 1
    print('------------------------')
    
    # print(str(graph))
    for n in graph.nodes: print(graph.nodes[n])
    print('')
    print(' '.join([graph.nodes[n]['form'] for n in graph.nodes if n]))
    
    graph.draw_graph()
    
    collocations, = extract_something(graph, collection_id, collocations )
    
    for c in collocations:
        print(c, collocations[c])
    #if count > 10:
    #    break
    
    if not collection_id == 0 and not collection_id % BATCH_SIZE:
        my_sqlite_db.save_coll_to_db(collocations, collection_id)
        collocations = {}
    
    
    print()
    


# saving last batch
my_sqlite_db.save_coll_to_db(collocations, collection_id)
collocations = {}
my_sqlite_db.index_fields()


TSV lines:   0%|          | 0/518 [00:00<?, ?it/s]

data/balanced_test.vert_10
------------------------
{'id': 1, 'lemma': 'Tallinn', 'pos': 'H', 'deprel': 'nmod', 'form': 'Tallinna', 'feats': {'prop': 'prop', 'sg': 'sg', 'gen': 'gen'}, 'verbform': None, 'head': 2}
{'id': 2, 'lemma': 'sotsiaal', 'pos': 'S', 'deprel': 'nmod', 'form': 'sotsiaal-', 'feats': {'com': 'com', 'sg': 'sg', 'nom': 'nom'}, 'verbform': None, 'head': 5}
{'id': 5, 'lemma': 'juhataja', 'pos': 'S', 'deprel': 'nsubj', 'form': 'juhataja', 'feats': {'com': 'com', 'sg': 'sg', 'nom': 'nom'}, 'verbform': None, 'head': 8}
{'id': 3, 'lemma': 'ja', 'pos': 'J', 'deprel': 'cc', 'form': 'ja', 'feats': {'sub': 'sub', 'crd': 'crd'}, 'verbform': None, 'head': 4}
{'id': 4, 'lemma': 'tervishoiuamet', 'pos': 'S', 'deprel': 'conj', 'form': 'tervishoiuameti', 'feats': {'com': 'com', 'sg': 'sg', 'gen': 'gen'}, 'verbform': None, 'head': 2}
{'id': 8, 'lemma': 'kinnitama', 'pos': 'V', 'deprel': 'root', 'form': 'kinnitas', 'feats': {'mod': 'mod', 'indic': 'indic', 'impf': 'impf', 'ps3': 'ps3',

In [ ]:
collection_id